# RTE+RRTMGP-NN training

*Last edited: 2024-09-24*

This Notebook generates files containing the neural network (NN) model that is later used in the RTE+RRTMGP-NN model. The implementation uses TensorFlow and Python, and Fortran routines are used to generate the training data set. The idea is to replace the RRTMGP lookup tables with NN. The code and data from Ukkonen & Hogan (2023) need to be installed for the Notebook to work. And after training, the model is saved to disk using two file formats, HDF5 and netCDF. The main code (RTE+RRTMGP) is written in Fortan, including the part that makes the prediction from the NN model.

Based on:

- Ukkonen, P., & Hogan, R. J. (2023). Implementation of a machine-learned gas optics parameterization in the ECMWF Integrated Forecasting System: RRTMGP-NN 2.0. Geoscientific Model Development, 16(11), 3241–3261. https://doi.org/10.5194/gmd-16-3241-2023

Data and code come from:

- Ukkonen, P. (2024). Peterukk/rte-rrtmgp-nn. https://github.com/peterukk/rte-rrtmgp-nn
- Ukkonen, P., Pincus, R., Hillman, B. R., Norman, M., fomics, & Heerwaarden, C. van. (2022). peterukk/rte-rrtmgp-nn: 2.0 (Version 2.0). Zenodo. https://doi.org/10.5281/zenodo.7413935
- Ukkonen, P. (2022). Code and extensive data for training neural networks for radiation, used in “Implementation of a machine-learned gas optics parameterization in the ECMWF Integrated Forecasting System: RRTMGP-NN 2.0”" [Dataset]. Zenodo. https://doi.org/10.5281/zenodo.7413952
- Ukkonen, P. (2022). Optimized version of the ecRad radiation scheme with new RRTMGP-NN gas optics. Zenodo. https://doi.org/10.5281/zenodo.7852526

Notes:

- The documentation is described in the work of Ukkonen & Hogan.
- In this Notebook, the base directory where the code and data are located is `ukk23test01/`.
- The training, which uses TensorFlow, is located in the `examples/rrtmgp-nn-training/` directory.
- This Notebook itself is located outside the base directory, one level below.
- Training data is generated from RRTMGP.
- To perform the prediction, "g-point" vectors are used, containing:
    - LW
        - Planck fraction, absorption cross-section, or both
    - SW
        - Absorption cross-section, or Rayleigh cross-section
- Models are saved to `../../neural/data` with a file name containing the custom radiation scores.

Dataset:

Due to space constraints, the datasets are not hosted on this GitHub repository, and need to be downloaded from the links above. Additionally, some data files are created during the execution of the routines, such as the NN training dataset. The data directories are:

- neural/data
- rrtmgp/data
- examples/rfmip-clear-sky/data
- examples/rfmip-clear-sky/output_fluxes
- examples/rrtmgp-nn-training/data
- examples/rrtmgp-nn-training/inputs_to_RRTMGP

Fortran and dependencies:

```bash
apt install gfortran libopenblas-dev libnetcdf-dev libnetcdff-dev
```

## Go to work dir

In [1]:
%cd ukk23test01/examples/rrtmgp-nn-training/

/home/x/Documents/inpe/tese/Ukk23/ukk23@gh/ukk23test01/examples/rrtmgp-nn-training


## Build the Fortran code

Environment variables configuration:

In [2]:
%env FC=gfortran
%env FCFLAGS=-ffree-line-length-none -m64 -march=native -O3 -lcurl
%env NCHOME=/usr
%env NFHOME=/usr
%env BLASLIB=openblas

env: FC=gfortran
env: FCFLAGS=-ffree-line-length-none -m64 -march=native -O3 -lcurl
env: NCHOME=/usr
env: NFHOME=/usr
env: BLASLIB=openblas


In [2]:
! make

VAR="../../"
make: Nothing to be done for 'all'.


- Example of executable command format: `./rrtmgp_lw_gendata_rfmipstyle [block_size] [input file] [k-distribution file] [input-output file]`

Once built, the next step is to generate the training data.

### Block_size

- The block size is the number of columns to be computed at a time, and must be an integer such that the remainder of dividing `ncol*nexp` by `block_size` is zero.
- `block_size = 3` worked in all cases:

## Training data generation

Using Fortran executables

In [33]:
%%bash
./rrtmgp_lw_gendata_rfmipstyle \
    3 \
    inputs_to_RRTMGP/inputs_Garand_BIG.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g128-210809.nc \
    data/ml_training_lw_g128_Garand_BIG.nc

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] input_output file]
 input fileinputs_to_RRTMGP/inputs_Garand_BIG.nc                                                                                               
 ncol:          42 nexp:         322 nlay:          42
 Doing         4508 blocks of size            3
 Calculation uses gases: water_vapor ozone carbon_dioxide methane nitrous_oxide oxygen nitrogen cfc11 cfc12 carbon_monoxide carbon_tetrachloride hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 
 min of play   19.2751713     k_dist%get_press_min()   1.00518358    
 -------------------------------------------------------------------------
 starting clear-sky longwave computations
 Finished with computations!
 mean of flux_down is:   84.6862488    
 mean of flux_up is:   277.106110    
 -------------------------------------------------------------------------
 Attempting to save RRTMGP input/output to data/ml_training_lw_g128_Garand_BIG.nc                    

In [35]:
%%bash
./rrtmgp_lw_gendata_rfmipstyle \
    3 \
    inputs_to_RRTMGP/inputs_AMON_ssp245_ssp585_2054_2100.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g128-210809.nc \
    data/ml_training_lw_g128_AMON_ssp245_ssp585_2054_2100.nc

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] input_output file]
 input fileinputs_to_RRTMGP/inputs_AMON_ssp245_ssp585_2054_2100.nc                                                                             
 ncol:         420 nexp:         200 nlay:          19
 Doing        28000 blocks of size            3
 Calculation uses gases: water_vapor ozone carbon_dioxide methane nitrous_oxide oxygen nitrogen cfc11 cfc12 carbon_monoxide carbon_tetrachloride hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 
 min of play   100.000000     k_dist%get_press_min()   1.00518358    
 -------------------------------------------------------------------------
 starting clear-sky longwave computations
 Finished with computations!
 mean of flux_down is:   83.7984009    
 mean of flux_up is:   279.662506    
 -------------------------------------------------------------------------
 Attempting to save RRTMGP input/output to data/ml_training_lw_g128_AMON_ssp245_ssp585_2054_2100.nc  

In [36]:
%%bash
./rrtmgp_lw_gendata_rfmipstyle \
    3 \
    inputs_to_RRTMGP/inputs_CAMS_new_CKDMIPstyle.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g128-210809.nc \
    data/ml_training_lw_g128_CAMS_new_CKDMIPstyle.nc

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] input_output file]
 input fileinputs_to_RRTMGP/inputs_CAMS_new_CKDMIPstyle.nc                                                                                     
 ncol:        1000 nexp:          42 nlay:          60
 Doing        14000 blocks of size            3
 Calculation uses gases: water_vapor ozone carbon_dioxide methane nitrous_oxide oxygen nitrogen cfc11 cfc12 carbon_monoxide carbon_tetrachloride hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 
 min of play   10.0000000     k_dist%get_press_min()   1.00518358    
 -------------------------------------------------------------------------
 starting clear-sky longwave computations
 Finished with computations!
 mean of flux_down is:   94.9205475    
 mean of flux_up is:   257.382263    
 -------------------------------------------------------------------------
 Attempting to save RRTMGP input/output to data/ml_training_lw_g128_CAMS_new_CKDMIPstyle.nc          

In [37]:
%%bash
./rrtmgp_lw_gendata_rfmipstyle \
    3 \
    inputs_to_RRTMGP/inputs_CKDMIP-MM-Big.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g128-210809.nc \
    data/ml_training_lw_g128_CKDMIP-MMM-Big.nc

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] input_output file]
 input fileinputs_to_RRTMGP/inputs_CKDMIP-MM-Big.nc                                                                                            
 ncol:         243 nexp:          58 nlay:          52
 Doing         4698 blocks of size            3
 Calculation uses gases: water_vapor ozone carbon_dioxide methane nitrous_oxide oxygen nitrogen cfc11 cfc12 carbon_monoxide carbon_tetrachloride hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 
 min of play  0.504999995     k_dist%get_press_min()   1.00518358    
 -------------------------------------------------------------------------
 starting clear-sky longwave computations
 Finished with computations!
 mean of flux_down is:   39.1128159    
 mean of flux_up is:   281.537994    
 -------------------------------------------------------------------------
 Attempting to save RRTMGP input/output to data/ml_training_lw_g128_CKDMIP-MMM-Big.nc                

## Training using TensorFlow

Loading the libraries:

In [109]:
import os
import sys
from sys import getsizeof as sizeof
import numpy as np

In [126]:
import tensorflow as tf
from tensorflow.keras import losses, optimizers
from tensorflow.keras.utils import Sequence

Checks GPU availability:

In [62]:
print(tf.config.list_physical_devices("GPU"))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Load routines contained in code files in the working directory:

In [71]:
from ml_load_save_preproc import (
    save_model_netcdf,
    load_rrtmgp,
    scale_outputs_wrapper,
    preproc_pow_standardization_reverse,
    preproc_tau_to_crossection,
    preproc_minmax_inputs_rrtmgp,
)

In [72]:
from ml_scaling_coefficients import xcoeffs_all, input_names_all

In [73]:
from ml_trainfuncs_keras import (
    create_model_mlp,
    expdiff,
    hybrid_loss_wrapper,
)

## Configure predictand, NN complexity, etc

In [74]:
predictand = "lw_both"

In [75]:
scaling_method = "Ukkonen2020"  # only option currently

For `use_existing_input_scaling_coefficients` True is generally a safe choice, min max coefficients have been computed using a large dataset spanning both LGM (Last Glacial Maximum) and high future emissions scenarios. However, check that your scaled inputs fall somewhere in the 0-1 range. Negative values in particular might cause problems:

In [76]:
use_existing_input_scaling_coefficients = True

## Loss function, metrics

In [77]:
patience = 70
epochs = 200

In [78]:
lossfunc = losses.MeanSquaredError
mymetrics = ["mean_absolute_error"]
expfirst = False

## Batch size and learning rate

In [128]:
lr = 0.001
batch_size = 2048

## NN Hyperparameters

Number of neurons in each hidden layer:

In [80]:
neurons = [72, 72]

Activation functions used after each layer: first the input layer, and then the hidden layers:

In [81]:
activ = ["softsign", "softsign", "linear"]

In [82]:
if np.size(activ) != np.size(neurons) + 1:
    print("Number of activations must be number of hidden layers + 1!")

Weight initializer: the default is probably an OK choice  (glorot):

In [83]:
initializer = "glorot_uniform"

## Routine for concatenating existing datasets containing raw inputs and outputs

In [84]:
def add_dataset(fpath, predictand, expfirst, x, y, col_dry, input_names, kdist,
                data_str):
    x_new, y_new, col_dry_new, input_names_new, kdist_new = load_rrtmgp(
        fpath, predictand, expfirst=expfirst)
    if not (kdist == kdist_new):
        print("Kdist does not match previous dataset!")
        return None
    if not (input_names == input_names_new):
        print("Input_names does not match previous dataset!")
        return None
    ns = x.shape[0]
    x = np.concatenate((x, x_new), axis=0)
    y = np.concatenate((y, y_new), axis=0)
    col_dry = np.concatenate((col_dry, col_dry_new), axis=0)
    print("{:.2e} samples previously, {:.2e} after adding data from: {}".format(
        ns, x.shape[0],
        fpath.split("/")[-1]))
    data_str = data_str + " , " + fpath.split("/")[-1]
    return x, y, col_dry, data_str

## Provide data containing inputs and outputs

- Profiles used:
    - Expanded Garand
    - GCM data (AMON_...)
    - CAMS data
    - Extended CKDMIP-Average-Maximum-Minimum profiles
- RFMIP ised used for validation.

The full dataset consumes a lot of RAM and VRAM

In [85]:
datadir = "/home/x/data-aux/"
fpath = datadir + "ml_training_lw_g128_Garand_BIG.nc"  # 0.6 GB
fpath2 = datadir + "ml_training_lw_g128_AMON_ssp245_ssp585_2054_2100.nc"  # 1.7 GB
fpath3 = datadir + "ml_training_lw_g128_CAMS_new_CKDMIPstyle.nc"  # 2.6 GB
fpath4 = datadir + "ml_training_lw_g128_CKDMIP-MMM-Big.nc"  # 0.8 GB

Load only the small ones, to fit RAM

Choose a small dataset just for testing (training using the full dataset requires a lot of RAM):

In [86]:
#fpaths = [fpath, fpath2, fpath4]  # rodou no ACER
fpaths = [fpath, fpath2, fpath3, fpath4]

ACER: GPU:0 with 3507 MB memory:  -> device: 0, name: NVIDIA GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0

## Load data 

Load training data

In [87]:
x_tr_raw, y_tr_raw, col_dry_tr, input_names, kdist = load_rrtmgp(
    fpaths[0], predictand, expfirst=expfirst)

input_names found in file
there are 13524 profiles in this dataset (322 experiments, 42 columns)


In [88]:
data_str = fpath.split("/")[-1]
# data_str

The full training dataset is split into multiple files:

In [89]:
%%time
# We can have different datasets that we merge
for fpath in fpaths[1:]:
    x_tr_raw, y_tr_raw, col_dry_tr, data_str = add_dataset(
        fpath,
        predictand,
        expfirst,
        x_tr_raw,
        y_tr_raw,
        col_dry_tr,
        input_names,
        kdist,
        data_str,
    )

input_names found in file
there are 84000 profiles in this dataset (200 experiments, 420 columns)
5.68e+05 samples previously, 2.16e+06 after adding data from: ml_training_lw_g128_AMON_ssp245_ssp585_2054_2100.nc
input_names found in file
there are 42000 profiles in this dataset (42 experiments, 1000 columns)
2.16e+06 samples previously, 4.68e+06 after adding data from: ml_training_lw_g128_CAMS_new_CKDMIPstyle.nc
input_names found in file
there are 14094 profiles in this dataset (58 experiments, 243 columns)
4.68e+06 samples previously, 5.42e+06 after adding data from: ml_training_lw_g128_CKDMIP-MMM-Big.nc
CPU times: user 3.58 s, sys: 5.44 s, total: 9.02 s
Wall time: 9.04 s


In [90]:
nx = x_tr_raw.shape[1]  # temperature + pressure + gases
ny = y_tr_raw.shape[1]  # number of g-points

In [91]:
shuffle = True

It can be a cell that consumes a lot of time and memory, depending on the dataset:

## Input and output scaling

In [92]:
%%time
if scaling_method != "Ukkonen2020":
    print("Only one type of pre-processing currently supported!")
else:
    # Input scaling - min-max
    if use_existing_input_scaling_coefficients:
        if xcoeffs_all == None:
            sys.exit("Input scaling coefficients (xcoeffs) missing!")
        (xmin_all, xmax_all) = xcoeffs_all
        # input_names loaded from file, describes inputs in order of x_tr_raw
        # input_names_all corresponds to xmin_all and xmax_all
        # Order of inputs may be different than in the existing coefficients,
        # account for that by indexing
        a = np.array(input_names_all)
        b = np.array(input_names)
        indices = np.where(b[:, None] == a[None, :])[1]
        xmin = xmin_all[indices]
        xmax = xmax_all[indices]
        x_tr = preproc_minmax_inputs_rrtmgp(x_tr_raw, (xmin, xmax))
    else:
        x_tr, xmin, xmax = preproc_minmax_inputs_rrtmgp(x_tr_raw)
        # Output scaling
        # first, do y = y / N if y is optical depth, to get cross-sections
        # then, square root scaling y: y=y**(1/nfac); cheaper and weaker version of
        # log scaling. nfac = 8 for cross-sections, 2 for Planck fraction
        # After this, use standard-scaling (not for Planck fraction)

    y_tr, ymean, ystd = scale_outputs_wrapper(y_tr_raw, col_dry_tr, predictand)

CPU times: user 2min 4s, sys: 14.1 s, total: 2min 18s
Wall time: 1min 11s


## I/O

RRTMGP-NN models are saved as NetCDF files which contain metadata describing how to obtain the physical outputs, as well as the training data

In [93]:
x_scaling_str = (
    "To get the required NN inputs, do the following: "
    "x(i) = log(x(i)) for i=pressure; "
    "x(i) = x(i)**(1/4) for i=H2O and O3; "
    "x(i) = (x(i) - xmin(i)) / (xmax(i) - xmin(i)) for all inputs"
)
y_scaling_str = (
    "Model predicts scaled cross-sections. Given the raw NN output y,"
    " do the following to obtain optical depth: "
    "y(igpt,j) = ystd(igpt)*y(igpt,j) + ymean(igpt); y(igpt,j) "
    "= y(igpt,j)**8; y(igpt,j) = y(igpt,j) * layer_dry_air_molecules(j)"
)

In [94]:
model_str = ""

Try to reduce memory consumption:

In [95]:
import gc

In [96]:
gc.collect()

427

## TensorFlow Training

Create and compile model

In [130]:
devstr = "/gpu:0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

optim = optimizers.Adam(learning_rate=lr)

model = create_model_mlp(nx=nx,
                         ny=ny,
                         neurons=neurons,
                         activ=activ,
                         kernel_init=initializer)

model.compile(loss=lossfunc, optimizer=optim, metrics=mymetrics)

## Start training

In [136]:
print(f"{sizeof(x_tr)/1024**3:.1f} GB")
print(f"{sizeof(y_tr)/1024**3:.1f} GB")

0.0 GB
5.2 GB


In [132]:
class DataGenerator(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y


train_gen = DataGenerator(x_tr, y_tr, batch_size)

Ref.: https://stackoverflow.com/questions/62916904/failed-copying-input-tensor-from-cpu-to-gpu-in-order-to-run-gatherve-dst-tensor

Time consuming part (36 min. approx.):

In [134]:
%%time
with tf.device(devstr):
    # history = model.fit(
    #     x_tr,
    #     y_tr,
    #     epochs=epochs,
    #     batch_size=batch_size,
    #     shuffle=shuffle,
    #     verbose=1,
    #     callbacks=[],
    # )
    history = model.fit(train_gen, epochs=epochs)
    history = history.history

Epoch 1/200


/home/x/conda/envs/tf2/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2645/2645 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.1036 - mean_absolute_error: 0.1594
Epoch 2/200
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0122 - mean_absolute_error: 0.0521
Epoch 3/200
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0081 - mean_absolute_error: 0.0418
Epoch 4/200
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0060 - mean_absolute_error: 0.0342
Epoch 5/200
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0049 - mean_absolute_error: 0.0306
Epoch 6/200
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0040 - mean_absolute_error: 0.0268
Epoch 7/200
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0039 - mean_absolute_error: 0.0269
Epoch 8/200
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0033 - mean_absolute_error: 0.0247
Epoch 9/200
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0030 - mean_absolute_error: 0.0231
Epoch 10/200
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0030 - mean_absolute_error: 0.0237
E

## Total parameters:

In [135]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 72)             │         1,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 256)            │        18,688 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,938 (296.64 KB)

 Trainable params: 25,312 (98.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 50,626 (197.76 KB)

## Save model

- Save model to NN model directory `../../neural/data` after training.
- File name includes loss values, so shouldn't override anything.

### Get a descriptive filename for the model

In [139]:
comm = "test01"
neurons_str = (np.array2string(np.array(neurons)).strip("[]").replace(" ", "_"))
source = kdist[12:].strip(".nc")
fpath_keras = ("../../neural/data/" + source + "_" + predictand[3:] + "_" +
               neurons_str + "_" + comm + ".h5")
fpath_netcdf = fpath_keras[:-3] + ".nc"

In [141]:
fpath_keras

'../../neural/data/lw-g128-210809_both_72_72_test01.h5'

In [140]:
fpath_netcdf

'../../neural/data/lw-g128-210809_both_72_72_test01.nc'

### Saving model in both netCDF and HDF5 format

In [ ]:
# model.save(fpath_keras, save_format="h5")

In [42]:
save_model_netcdf(fpath_netcdf,
                  model,
                  activ,
                  input_names,
                  kdist,
                  xmin,
                  xmax,
                  ymean,
                  ystd,
                  y_scaling_comment=y_scaling_str,
                  x_scaling_comment=x_scaling_str,
                  data_comment=data_str,
                  model_comment=model_str)

## References

Ukkonen, P., & Hogan, R. J. (2023). Implementation of a machine-learned gas optics parameterization in the ECMWF Integrated Forecasting System: RRTMGP-NN 2.0. Geoscientific Model Development, 16(11), 3241–3261. https://doi.org/10.5194/gmd-16-3241-2023

Ukkonen, P., & Hogan, R. J. (2024). Twelve Times Faster yet Accurate: A New State-Of-The-Art in Radiation Schemes via Performance and Spectral Optimization. Journal of Advances in Modeling Earth Systems, 16(1), e2023MS003932. https://doi.org/10.1029/2023MS003932

Ukkonen, P., & Hogan, R. J. (2023). Fast computation of cloud 3D radiative effects in dynamical models by optimizing the ecRad scheme [Preprint]. Preprints. https://doi.org/10.22541/essoar.168298700.07329865/v1

Ukkonen, P. (2022). Improving the trade-off between accuracy and efficiency of atmospheric radiative transfer computations by using machine learning and code optimization. http://dx.doi.org/10.13140/RG.2.2.27880.03846

Ukkonen, P. (2022). Exploring Pathways to More Accurate Machine Learning Emulation of Atmospheric Radiative Transfer. Journal of Advances in Modeling Earth Systems, 14(4), e2021MS002875. https://doi.org/10.1029/2021MS002875

Yao, Y., Zhong, X., Zheng, Y., & Wang, Z. (2023). A Physics-Incorporated Deep Learning Framework for Parameterization of Atmospheric Radiative Transfer. Journal of Advances in Modeling Earth Systems, 15(5), e2022MS003445. https://doi.org/10.1029/2022MS003445

## Conda environment

In [ ]:
%%bash
source ${HOME}/conda/bin/activate tf2
conda export --file ukk23test01-train-02.yml

## Data

In [ ]:
%%bash
ls -1 ukk23test01/neural/data/ > neur_data.txt
ls -1 ukk23test01/rrtmgp/data/ > rrtm_data.txt
ls -1 ukk23test01/examples/rfmip-clear-sky/data/ > exam_rfmi_data.txt
ls -1 ukk23test01/examples/rfmip-clear-sky/output_fluxes/ > exam_rfmi_flux.txt
ls -1 ukk23test01/examples/rrtmgp-nn-training/data/ > exam_rrtm_data.txt
ls -1 ukk23test01/examples/rrtmgp-nn-training/inputs_to_RRTMGP/ >  exam_rrtm_rrtm.txt